In [91]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import random
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression


In [167]:
train_df = pd.read_csv("train.csv")
x_train = train_df.drop(["SalePrice"], axis = 1)
y_train = train_df["SalePrice"]
x_test = pd.read_csv('test_toup.csv')

answer_df = pd.read_csv("samples submission.csv")

In [168]:
y_true = np.array(answer_df["SalePrice"])
y_true

array([208500, 181500, 223500, 140000, 250000, 143000, 307000, 200000,
       129900, 118000, 129500, 345000, 144000, 279500, 157000, 132000,
       149000,  90000, 159000, 139000, 325300, 139400, 230000, 129900,
       154000, 256300, 134800, 306000, 207500,  68500,  40000, 149350,
       179900, 165500, 277500, 309000, 145000, 153000, 109000,  82000,
       160000, 170000, 144000, 130250, 141000, 319900, 239686, 249700,
       113000, 127000, 177000, 114500, 110000, 385000, 130000, 180500,
       172500, 196500, 438780, 124900, 158000, 101000, 202500, 140000,
       219500, 317000, 180000, 226000,  80000, 225000, 244000, 129500,
       185000, 144900, 107400,  91000, 135750, 127000, 136500, 110000,
       193500, 153500, 245000, 126500, 168500, 260000, 174000, 164500,
        85000, 123600, 109900,  98600, 163500, 133900, 204750, 185000,
       214000,  94750,  83000, 128950, 205000, 178000, 118964, 198900,
       169500, 250000, 100000, 115000, 115000, 190000, 136900, 180000,
      

In [169]:
x_train.shape

(1321, 80)

In [170]:
x_train["type"] = "train"
x_test["type"] = "test"

In [171]:
x_whole = pd.concat([x_train, x_test], axis = 0, ignore_index=True)
print(x_train.shape)
print(x_test.shape)
print(x_whole.shape)

(1321, 81)
(139, 81)
(1460, 81)


In [172]:
object_dtype = []

for i in x_whole.columns:
    if x_whole[i].dtype == 'object':
        object_dtype.append(i)
        
np.array(object_dtype)

array(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour',
       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish',
       'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence',
       'MiscFeature', 'SaleType', 'SaleCondition', 'type'], dtype='<U13')

In [173]:
x_whole['MasVnrType'].unique()

array(['None', 'BrkFace', 'Stone', 'BrkCmn', nan], dtype=object)

In [174]:
valid_na_columns = ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']

In [175]:
num_but_obj_columns = ['MSSubClass', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd','GarageCars', 'Fireplaces', 'GarageYrBlt', 'MoSold', 'YrSold']

In [176]:
#for int and float columns
meanable_cols = []


for i in x_whole.columns:
    if i not in num_but_obj_columns:
        if x_whole[i].dtypes == 'int' or x_whole[i].dtypes == 'float':
            meanable_cols.append(i)
            
meanable_cols

['Id',
 'LotFrontage',
 'LotArea',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal']

In [177]:
le = preprocessing.LabelEncoder()
ohe = OneHotEncoder() 
for i in x_whole.columns:
    if i not in valid_na_columns:
        if i in meanable_cols:
            x_whole[i].fillna(-1, inplace = True)
            for j in range(len(train_df[i])):
                if train_df[i][j] < 0:
                    x_whole[i][j] = random.choice([x_whole[i].mean() - x_whole[i].std(), x_whole[i].mean() + x_whole[i].std()])
        else:
            print(i)
            x_whole[i].fillna("None", inplace = True)
            for j in range(len(x_whole[i])):
                if x_whole[i][j] == "None":
                    x_whole[i][j] = random.choice(x_whole[i].dropna().unique())
            if i in object_dtype:
                x_whole[i] = le.fit_transform(x_whole[i])
                
    else:
        print(i)
        x_whole[i].fillna('NaN', inplace=True)
        x_whole[i] = le.fit_transform(x_whole[i])

MSSubClass
MSZoning
Street
Alley
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
OverallQual
OverallCond
YearBuilt
YearRemodAdd
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
ExterQual
ExterCond
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Heating
HeatingQC
CentralAir
Electrical


/home/rohan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


BsmtFullBath
BsmtHalfBath
FullBath
HalfBath
BedroomAbvGr
KitchenAbvGr
KitchenQual
TotRmsAbvGrd
Functional
Fireplaces
FireplaceQu
GarageType
GarageYrBlt
GarageFinish
GarageCars
GarageQual
GarageCond
PavedDrive
PoolQC
Fence
MiscFeature
MoSold
YrSold
SaleType
SaleCondition
type


In [178]:
x_whole['GarageYrBlt'].unique()

array([1997.0, 2006.0, 2005.0, 2000.0, 1999.0, 1963.0, 2004.0, 1931.0,
       2001.0, 1926.0, 1936.0, 1975.0, 2007.0, 1971.0, 1960.0, 1923.0,
       1950.0, 2009.0, 1984.0, 2003.0, 1929.0, 1964.0, 1955.0, 1981.0,
       1991.0, 1987.0, 1961.0, 1986.0, 1988.0, 1958.0, 1935.0, 1920.0,
       1957.0, 1993.0, 1990.0, 1916.0, 1979.0, 1932.0, 1972.0, 1989.0,
       1976.0, 1918.0, 1980.0, 1924.0, 1948.0, 1962.0, 1996.0, 1977.0,
       1940.0, 1939.0, 2002.0, 1995.0, 1974.0, 1959.0, 1949.0, 2008.0,
       1994.0, 1910.0, 1954.0, 1956.0, 1978.0, 1965.0, 1966.0, 1982.0,
       1951.0, 1992.0, 1970.0, 1908.0, 1953.0, 1968.0, 1998.0, 1925.0,
       1983.0, 1941.0, 1930.0, 2010.0, 1985.0, 1927.0, 1947.0, 1937.0,
       1973.0, 1906.0, 1942.0, 1915.0, 1938.0, 1967.0, 1952.0, 1928.0,
       1922.0, 1934.0, 1914.0, 1969.0, 1921.0, 1945.0, 1946.0, 1933.0,
       1900.0], dtype=object)

In [179]:
x_whole['GarageYrBlt'] = x_whole['GarageYrBlt'].astype(float)
x_whole['GarageYrBlt'].unique()

array([1997., 2006., 2005., 2000., 1999., 1963., 2004., 1931., 2001.,
       1926., 1936., 1975., 2007., 1971., 1960., 1923., 1950., 2009.,
       1984., 2003., 1929., 1964., 1955., 1981., 1991., 1987., 1961.,
       1986., 1988., 1958., 1935., 1920., 1957., 1993., 1990., 1916.,
       1979., 1932., 1972., 1989., 1976., 1918., 1980., 1924., 1948.,
       1962., 1996., 1977., 1940., 1939., 2002., 1995., 1974., 1959.,
       1949., 2008., 1994., 1910., 1954., 1956., 1978., 1965., 1966.,
       1982., 1951., 1992., 1970., 1908., 1953., 1968., 1998., 1925.,
       1983., 1941., 1930., 2010., 1985., 1927., 1947., 1937., 1973.,
       1906., 1942., 1915., 1938., 1967., 1952., 1928., 1922., 1934.,
       1914., 1969., 1921., 1945., 1946., 1933., 1900.])

In [180]:
le.inverse_transform(x_whole["type"])

array(['train', 'train', 'train', ..., 'test', 'test', 'test'],
      dtype=object)

In [181]:
x_whole.drop("LotFrontage", axis = 1, inplace = True)

In [184]:
x_whole["Years_exist"] = x_whole["YearBuilt"] - x_whole["YrSold"]
x_whole.drop("YearBuilt", axis=1, inplace=True)

In [185]:
x_train = x_whole[x_whole["type"] == 1]
x_test = x_whole[x_whole["type"] == 0]
print(x_train.shape)
print(x_test.shape)
x_train.drop("type", axis = 1, inplace=True)
x_test.drop("type", axis = 1, inplace = True)

(1321, 80)
(139, 80)


/home/rohan/.local/lib/python3.6/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [186]:
# model = XGBClassifier()

In [187]:
# model.fit(x_train, y_train)

In [188]:
model = LinearRegression()
model.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [189]:
y_pred = model.predict(x_test)

In [190]:
y_pred

array([215418.84685237, 205148.16797306, 217336.1107974 , 171825.33689319,
       290596.86665237, 169038.76026272, 287389.57329719, 236663.46377068,
       164639.53149055, 120168.41325519, 125535.97267834, 385126.37594206,
       138563.4758885 , 252595.63519611, 152841.52892671, 128519.54032172,
       153854.77559708,  97921.11960494, 149921.24853885, 110474.17861793,
       310712.3865141 , 121855.8945825 , 239759.67978066, 135271.04547454,
       157037.81846458, 263453.3663406 , 132198.27354733, 277289.31337442,
       186534.88177074,  30676.47486067,  89658.55439104, 128031.82147461,
       216964.4284016 , 170685.74230252, 295520.32176521, 308877.71129075,
       133360.74028692, 161586.06532456, 117070.65244836,  86093.29395199,
       136386.49022173, 154057.63382822, 163658.56995367, 135925.75016086,
       136604.22629004, 298861.38093232, 267205.6610953 , 250999.86131622,
        55282.75661785, 130313.58685891, 178534.86145279, 111823.7898753 ,
        93816.13743607, 3

In [191]:
df_out = pd.DataFrame(y_pred)

In [192]:
df_out


,0
0,215418.846852
1,205148.167973
2,217336.110797
3,171825.336893
4,290596.866652
...,...
134,158989.413621
135,198890.673606
136,131445.217733
137,172726.866110


In [193]:
df_out.columns = ["SalePrice"]

In [194]:
df_out["Id"] = df_out.index

In [195]:
df_out = df_out[["Id", "SalePrice"]]

In [196]:
df_out


,Id,SalePrice
0,0,215418.846852
1,1,205148.167973
2,2,217336.110797
3,3,171825.336893
4,4,290596.866652
...,...,...
134,134,158989.413621
135,135,198890.673606
136,136,131445.217733
137,137,172726.866110


In [197]:
df_out["Id"] = range(1, 140)
df_out["SalePrice"] = df_out["SalePrice"].astype(int)

In [198]:
df_out.set_index("Id")

,SalePrice
Id,
1,215418
2,205148
3,217336
4,171825
5,290596
...,...
135,158989
136,198890
137,131445


In [199]:
df_out.to_csv("ouput.csv", index = False)

In [201]:
print((metrics.mean_squared_error(y_true, df_out["SalePrice"])))

490582910.46043164
